In [1]:
import tensorflow as tf
import numpy as np
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape((-1, 28*28))
X_test = X_train.reshape((-1, 28*28))
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

n_inputs = X_train.shape[1]

[dset.shape for dset in [X_train, y_train, X_test, y_test]]

C:\Users\tdele\Anaconda3\envs\ml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[(60000, 784), (60000,), (60000, 784), (10000,)]

8.a.

In [2]:
from functools import partial

tf.reset_default_graph()

layers_neurons = [100 for _ in range(5)]

X = tf.placeholder(tf.float32, (None, n_inputs), name='X')
y = tf.placeholder(tf.int32, (None), name='y')

he_init = tf.variance_scaling_initializer()

with tf.name_scope('dnn'):
    layer = X
    for i, neurons in enumerate(layers_neurons):
        hidden_layer = tf.layers.dense(layer, neurons, activation=tf.nn.elu, kernel_initializer=he_init, name='hidden{}'.format(i))
        layer = hidden_layer
    logits = tf.layers.dense(layer, 5, kernel_initializer=he_init, name='logits')
    y_proba = tf.nn.softmax(logits, name='softmax')
        
with tf.name_scope('loss'):
    xent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='cross_entropy')
    loss = tf.reduce_mean(xent, name='loss')
    
with tf.name_scope('optimize'):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('metrics'):
    precision = tf.metrics.precision(logits, y, name='precision')
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

8.b.

In [3]:
X_train04 = X_train[y_train < 5]
y_train04 = y_train[y_train < 5]
X_train59 = X_train[y_train > 4]
y_train59 = y_train[y_train > 4]

In [4]:
from support.neural_nets import *

epochs = 50
batch_size = 100
# n_batches = int(np.ceil(X_train04.shape[0]/batch_size))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for X_batch, y_batch in fetch_batch(X_train04, y_train04, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch+1 % 10 == 0:
            print("Epoch {}")
            for X_batch, y_batch in fetch_batch(X_train04, y_train04):
                sess.eval(precision, feed_dict={X: X_batch, y: y_batch})
            saver.save(sess, './tmp/dl/dl_ckpt_{}.ckpt'.format(epoch))
    saver.save(sess, './tmp/dl/dl_final_model.ckpt')